In [1]:
import os, tarfile
import re
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
WM_PATH = "/data09/shared/pdutta_data/Matthew_data/Splice_sites/Raw_data/DNA_bert_splicesites_w_mutations.tsv"
WOM_PATH = "/data09/shared/pdutta_data/Matthew_data/Splice_sites/Raw_data/DNA_bert_splicesites_wo_mutations_nodup.tsv"
OUTPUT_FOLDER= "/data09/shared/pdutta_data/Matthew_data/Splice_sites/Fine_tuned_data"
POSTPROCESSING_FOLDER = "/data09/shared/pdutta_data/Matthew_data/Splice_sites/Postprocessing_data"

In [3]:
mutated_splice_site_csv  =  pd.read_csv(WM_PATH,  sep='\t')
print(mutated_splice_site_csv.shape)
without_mutated_splice_site_csv  =  pd.read_csv(WOM_PATH,  sep='\t')
print (without_mutated_splice_site_csv.shape)
print(mutated_splice_site_csv.head())
print(without_mutated_splice_site_csv.head())

(1513085, 4)
(432995, 3)
                                patient                region  \
0  0059fce3-8e42-4f93-a6e0-abe8525c05c0  chr1:1668127-1668527   
1  0059fce3-8e42-4f93-a6e0-abe8525c05c0  chr1:1754344-1754744   
2  0059fce3-8e42-4f93-a6e0-abe8525c05c0  chr1:1825197-1825597   
3  0059fce3-8e42-4f93-a6e0-abe8525c05c0  chr1:2526264-2526664   
4  0059fce3-8e42-4f93-a6e0-abe8525c05c0  chr1:2591732-2592132   

                                            sequence  class  
0  TGATTAGTAAATGGAAAATGGAAACCGATAGAGTACCCTTTTGTTT...      1  
1  CACTGACTTCCCTGATGCATGCCCTCTGACTCCTGCTGGACCTCAG...      1  
2  CCACCACGCCCACCTTAAACTTTCAATTTACTTTTCTCACCTGCTT...      1  
3  TATGCGAGGGGGCGTTTGGCAGTCAAATATATGCGCATATATCTCC...      1  
4  CACAGGGGTAGGTCCTGGAGCCAGACCTCTGCTCTGCCCTGAGATC...      1  
             region                                           sequence  class
0  chr1:65219-65619  TTTATTATATTTGTTTTGCTGTGATTCTTTAAAAAGCACCTTTAGA...      0
1  chr1:65233-65633  TTTGCTGTGATTCTTTAAAAAGCACCTTTAGACTC

In [4]:
len(without_mutated_splice_site_csv.iloc[105419]['sequence'])

400

In [8]:
df_test = mutated_splice_site_csv['sequence']
df_test.drop_duplicates()

0          TGATTAGTAAATGGAAAATGGAAACCGATAGAGTACCCTTTTGTTT...
1          CACTGACTTCCCTGATGCATGCCCTCTGACTCCTGCTGGACCTCAG...
2          CCACCACGCCCACCTTAAACTTTCAATTTACTTTTCTCACCTGCTT...
3          TATGCGAGGGGGCGTTTGGCAGTCAAATATATGCGCATATATCTCC...
4          CACAGGGGTAGGTCCTGGAGCCAGACCTCTGCTCTGCCCTGAGATC...
                                 ...                        
1513076    ACTGCAACAATCTTTCTTTTTTATTCTGCTTTGTAGTTGTGTTTGT...
1513077    TACTAGTGGTTCGCTACAGGAGGAACGTGAAAAGAACATCTCCAGA...
1513078    GCTGACGTGCACATACATCCCAGCTCTGCATCCCGAAATGCTGCAA...
1513083    CAGTGACCAGCGGGCGATGGTCTCTGGCTTGAGGGACAGGATGGAG...
1513084    AGGCCCGCCCCCGCGCGCCAGCCGCGCGCCCGCTGCCCCCTCCTCG...
Name: sequence, Length: 1104469, dtype: object

In [5]:
without_mutated_splice_site_csv[without_mutated_splice_site_csv.sequence.str.contains(r'[N]')]

,region,sequence,class
44450,chr10:58267694-58268094,CTCAGGCTCGGGCGCGAGGAGGCCCGGGGGTTCCCGCGGCTGGTGC...,0
105419,chr12:123475893-123476293,AAAATTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCACTGT...,0
113169,chr13:113722874-113723274,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
113170,chr13:113722957-113723357,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
198028,chr18:46969712-46970112,CCCAGCAGAGGCCCGCCCTGAGCGGCGTGAGCGTGGCCTCTTCCGG...,0
...,...,...,...
432990,chrY:57211694-57212094,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
432991,chrY:57211874-57212274,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
432992,chrY:57211983-57212383,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0
432993,chrY:57211986-57212386,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,0


In [6]:
without_mutated_splice_site_csv = without_mutated_splice_site_csv.drop(without_mutated_splice_site_csv[without_mutated_splice_site_csv.sequence.str.contains(r'[N]')].index).reset_index(drop=True)
mutated_splice_site_csv = mutated_splice_site_csv.drop(mutated_splice_site_csv[mutated_splice_site_csv.sequence.str.contains(r'[N]')].index).reset_index(drop=True)

In [7]:
without_mutated_splice_site_csv

,region,sequence,class
0,chr1:65219-65619,TTTATTATATTTGTTTTGCTGTGATTCTTTAAAAAGCACCTTTAGA...,0
1,chr1:65233-65633,TTTGCTGTGATTCTTTAAAAAGCACCTTTAGACTCAGTGAGATAGC...,0
2,chr1:65320-65720,TCACTCAGGAAAATTCTGTGTGTTTTCTCTAATGGCCAAGGGAAAA...,0
3,chr1:65373-65773,GACTATAAAAGTTAGTCTCAGTACACAAAGCTCAGACTGGCTATTC...,0
4,chr1:68837-69237,CACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGGGAAG...,0
...,...,...,...
432437,chrY:25621962-25622362,TCAGTAAGTGAAATAATGGAGGTACATTATACAGGAGTAAAGGCTT...,0
432438,chrY:25623834-25624234,GTATTTTTGACTGGAACTTTCACCCAAGAGGTTATGATTCAAATTA...,0
432439,chrY:25624138-25624538,CCATCTTCATAGCCCAAAACAATTTCACCCATAGCTAAGGCTTGGA...,0
432440,chrY:25624255-25624655,GAGTAACAAAACAGCCTCTTTGCCCAAATGTGATTATTTTATGCAC...,0


In [8]:
mutated_splice_site_csv

,patient,region,sequence,class
0,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1668127-1668527,TGATTAGTAAATGGAAAATGGAAACCGATAGAGTACCCTTTTGTTT...,1
1,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1754344-1754744,CACTGACTTCCCTGATGCATGCCCTCTGACTCCTGCTGGACCTCAG...,1
2,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1825197-1825597,CCACCACGCCCACCTTAAACTTTCAATTTACTTTTCTCACCTGCTT...,1
3,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:2526264-2526664,TATGCGAGGGGGCGTTTGGCAGTCAAATATATGCGCATATATCTCC...,1
4,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:2591732-2592132,CACAGGGGTAGGTCCTGGAGCCAGACCTCTGCTCTGCCCTGAGATC...,1
...,...,...,...,...
1513080,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:152759643-152760043,GGAAGGCATCCTCTCAACCCTGGCCCAGAAGCCCTATCAAAGTGAA...,1
1513081,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:153770138-153770538,CTTCCATTGTGCGTTCGGGGACTATGACAGCTTGGCTCATGTGGAA...,1
1513082,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:153776014-153776414,CGTGGAGTTCCAGCCCCTGACGCTGATGGTGCTGGTGGGGCCCGGG...,1
1513083,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:154150456-154150856,CAGTGACCAGCGGGCGATGGTCTCTGGCTTGAGGGACAGGATGGAG...,1


In [9]:
def seq2kmer(seq, k):
    seq=seq.upper()
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [10]:
without_mutated_splice_site_csv['seq_kmer'] = without_mutated_splice_site_csv.apply(lambda row: seq2kmer(row['sequence'], 6), axis=1)
mutated_splice_site_csv['seq_kmer'] = mutated_splice_site_csv.apply(lambda row: seq2kmer(row['sequence'], 6), axis=1)

In [11]:
without_mutated_splice_site_csv

,region,sequence,class,seq_kmer
0,chr1:65219-65619,TTTATTATATTTGTTTTGCTGTGATTCTTTAAAAAGCACCTTTAGA...,0,TTTATT TTATTA TATTAT ATTATA TTATAT TATATT ATAT...
1,chr1:65233-65633,TTTGCTGTGATTCTTTAAAAAGCACCTTTAGACTCAGTGAGATAGC...,0,TTTGCT TTGCTG TGCTGT GCTGTG CTGTGA TGTGAT GTGA...
2,chr1:65320-65720,TCACTCAGGAAAATTCTGTGTGTTTTCTCTAATGGCCAAGGGAAAA...,0,TCACTC CACTCA ACTCAG CTCAGG TCAGGA CAGGAA AGGA...
3,chr1:65373-65773,GACTATAAAAGTTAGTCTCAGTACACAAAGCTCAGACTGGCTATTC...,0,GACTAT ACTATA CTATAA TATAAA ATAAAA TAAAAG AAAA...
4,chr1:68837-69237,CACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGGGAAG...,0,CACTGA ACTGAG CTGAGA TGAGAA GAGAAG AGAAGC GAAG...
...,...,...,...,...
432437,chrY:25621962-25622362,TCAGTAAGTGAAATAATGGAGGTACATTATACAGGAGTAAAGGCTT...,0,TCAGTA CAGTAA AGTAAG GTAAGT TAAGTG AAGTGA AGTG...
432438,chrY:25623834-25624234,GTATTTTTGACTGGAACTTTCACCCAAGAGGTTATGATTCAAATTA...,0,GTATTT TATTTT ATTTTT TTTTTG TTTTGA TTTGAC TTGA...
432439,chrY:25624138-25624538,CCATCTTCATAGCCCAAAACAATTTCACCCATAGCTAAGGCTTGGA...,0,CCATCT CATCTT ATCTTC TCTTCA CTTCAT TTCATA TCAT...
432440,chrY:25624255-25624655,GAGTAACAAAACAGCCTCTTTGCCCAAATGTGATTATTTTATGCAC...,0,GAGTAA AGTAAC GTAACA TAACAA AACAAA ACAAAA CAAA...


In [12]:
mutated_splice_site_csv

,patient,region,sequence,class,seq_kmer
0,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1668127-1668527,TGATTAGTAAATGGAAAATGGAAACCGATAGAGTACCCTTTTGTTT...,1,TGATTA GATTAG ATTAGT TTAGTA TAGTAA AGTAAA GTAA...
1,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1754344-1754744,CACTGACTTCCCTGATGCATGCCCTCTGACTCCTGCTGGACCTCAG...,1,CACTGA ACTGAC CTGACT TGACTT GACTTC ACTTCC CTTC...
2,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:1825197-1825597,CCACCACGCCCACCTTAAACTTTCAATTTACTTTTCTCACCTGCTT...,1,CCACCA CACCAC ACCACG CCACGC CACGCC ACGCCC CGCC...
3,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:2526264-2526664,TATGCGAGGGGGCGTTTGGCAGTCAAATATATGCGCATATATCTCC...,1,TATGCG ATGCGA TGCGAG GCGAGG CGAGGG GAGGGG AGGG...
4,0059fce3-8e42-4f93-a6e0-abe8525c05c0,chr1:2591732-2592132,CACAGGGGTAGGTCCTGGAGCCAGACCTCTGCTCTGCCCTGAGATC...,1,CACAGG ACAGGG CAGGGG AGGGGT GGGGTA GGGTAG GGTA...
...,...,...,...,...,...
1513080,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:152759643-152760043,GGAAGGCATCCTCTCAACCCTGGCCCAGAAGCCCTATCAAAGTGAA...,1,GGAAGG GAAGGC AAGGCA AGGCAT GGCATC GCATCC CATC...
1513081,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:153770138-153770538,CTTCCATTGTGCGTTCGGGGACTATGACAGCTTGGCTCATGTGGAA...,1,CTTCCA TTCCAT TCCATT CCATTG CATTGT ATTGTG TTGT...
1513082,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:153776014-153776414,CGTGGAGTTCCAGCCCCTGACGCTGATGGTGCTGGTGGGGCCCGGG...,1,CGTGGA GTGGAG TGGAGT GGAGTT GAGTTC AGTTCC GTTC...
1513083,ffa1d90c-4c1a-4b1a-824a-b313765817f9,chrX:154150456-154150856,CAGTGACCAGCGGGCGATGGTCTCTGGCTTGAGGGACAGGATGGAG...,1,CAGTGA AGTGAC GTGACC TGACCA GACCAG ACCAGC CCAG...


In [17]:
print(len(without_mutated_splice_site_csv.iloc[0]['sequence']))
print(len(without_mutated_splice_site_csv.iloc[0]['seq_kmer'].split()))

400
395


In [44]:
all_data = pd.concat([without_mutated_splice_site_csv[['seq_kmer', 'class']], mutated_splice_site_csv[['seq_kmer', 'class']]], ignore_index=True)
print(all_data.shape)

(1945527, 2)


In [53]:
all_data.to_csv(POSTPROCESSING_FOLDER+"/all_data.tsv", sep="\t", index=False)

In [46]:
all_data

,seq_kmer,class
0,TTTATT TTATTA TATTAT ATTATA TTATAT TATATT ATAT...,0
1,TTTGCT TTGCTG TGCTGT GCTGTG CTGTGA TGTGAT GTGA...,0
2,TCACTC CACTCA ACTCAG CTCAGG TCAGGA CAGGAA AGGA...,0
3,GACTAT ACTATA CTATAA TATAAA ATAAAA TAAAAG AAAA...,0
4,CACTGA ACTGAG CTGAGA TGAGAA GAGAAG AGAAGC GAAG...,0
...,...,...
1945522,GGAAGG GAAGGC AAGGCA AGGCAT GGCATC GCATCC CATC...,1
1945523,CTTCCA TTCCAT TCCATT CCATTG CATTGT ATTGTG TTGT...,1
1945524,CGTGGA GTGGAG TGGAGT GGAGTT GAGTTC AGTTCC GTTC...,1
1945525,CAGTGA AGTGAC GTGACC TGACCA GACCAG ACCAGC CCAG...,1


# Creating train and testing dataset

In [47]:
train, test = train_test_split(all_data, test_size=0.10)
train, val = train_test_split(train, test_size=0.10)
print (train.shape, test.shape, val.shape)
print(train.groupby(['class']).size(), test.groupby(['class']).size() , val.groupby(['class']).size())

(1575876, 2) (194553, 2) (175098, 2)
class
0     350130
1    1225746
dtype: int64 class
0     43142
1    151411
dtype: int64 class
0     39170
1    135928
dtype: int64


In [48]:
val.to_csv(OUTPUT_FOLDER+"/Evaluation_data/dev.tsv", sep="\t", index=False)
test.to_csv(OUTPUT_FOLDER+ "/dev.tsv", sep="\t", index=False)
train.to_csv(OUTPUT_FOLDER+"/train.tsv", sep="\t", index=False)